In [ ]:
from PIL import Image
import numpy as np
import scipy
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import sys
import os
from multiprocessing import Pool
from functools import partial
directory = 'figures'
if not os.path.exists(directory):
    os.makedirs(directory)
from concurrent.futures import ProcessPoolExecutor, as_completed
from dask.distributed import Client, progress
from dask import compute, delayed
import dask.array as da
from dask.diagnostics import ProgressBar
plt.rcParams['figure.dpi']=400

# Problem 4.2

In [ ]:
def sparseH(L, J, h, periodic):
    
    """
    generates the sparse Hamiltonian matrix for the quantum Ising chain
    
        Parameters:
            L (int): length of chain
            J (float): ising interaction strength
            h (float): magnetic field strength
            periodic (bool): does the chain have periodic boundary conditions?
            
        Returns:
            H (csr_matrix): sparse matrix representing the Hamiltonian operator
    """
    
    dim = 2 ** L # dimensions of the Hilbert space
    
    # initialize 
    H_data = []
    H_rows = []
    H_cols = []
    
    "Calculation of off-diagonal elements due to the magnetic field"
    
    for beta in range(dim): # iterate over all states
        
        for j in range(1, L + 1): # iterate over all sites
            
            alpha = beta ^ (1 << (j - 1)) # flips jth bit of beta to get the state alpha that is related to beta by a single bit flip
            
            "Keep track of the indices with non-zero matrix elements"
            
            H_data.append(-h)
            H_rows.append(alpha)
            H_cols.append(beta)
    
    "Calculation of diagonal elements due to Ising interaction"

    for alpha in range(dim):  # iterate over all states
        
        A = 0
        
        for j in range(1, L): # iterate over all sites
            
            if 2 * (alpha & (1 << (j - 1))) == alpha & (1 << j): # check if site j and j+1 have the same spin
                
                A -= J # if they do, decrease the energy by the ising interaction term
                
            else:
                
                A += J # if not, increase the energy by the ising interaction term
                
        "Handling periodic boundary conditions"
                
        if periodic and L > 1: # L > 1 needed for periodicity to mean anything
            
            if (alpha & (1 << (L - 1))) == ((alpha & (1 << 0)) * (2 ** (L - 1))): # Check if the states at either end have the same spin
                
                A -= J # if they do, decrease the energy by the ising interaction term
                
            else:
                
                A += J # if not, increase the energy by the ising interaction term
        
        if A != 0: # Check if the resulting matrix element is non-zero, if so, keep track of it
        
            H_data.append(A)
            H_rows.append(alpha)
            H_cols.append(alpha)

    H_data = np.array(H_data, dtype=float) # convert the list into a np array
    
    H = scipy.sparse.csr_matrix((H_data, (H_rows, H_cols)), shape=(dim, dim), dtype=np.float64) # make it into a csr sparse matrix
    
    return H

In [ ]:
def psi_gs(L, J, h, periodic):
    H = sparseH(L, J, h, periodic)
    return scipy.sparse.linalg.eigsh(H, k=1, which='SA')[1]

In [ ]:
def entanglement_entropy(l, L, J, h, periodic):

    vec = psi_gs(L, J, h, periodic)
    psi_matrix = vec.reshape((2**l, 2**(L-l)))
    u, s, vh = np.linalg.svd(psi_matrix, compute_uv=True)
    lambdas = s**2
    S = -np.sum(lambdas * np.log(lambdas))
    
    return S

In [ ]:
def compute_entropies(L_values, h, periodic):
    results = {}
    for L in L_values:
        ls = range(1, L)
        Entropies = []
        for l in tqdm(ls, desc=f'Computing for L={L}'):
            entropy = entanglement_entropy(l, L, 1, h, periodic)
            Entropies.append(entropy)
        results[L] = Entropies
    return results

def plot_results(results):
    plt.figure(figsize=(10, 6))
    for L, entropies in results.items():
        plt.plot(range(1, L)/np.array(L), entropies, label=f'L={L}')
    plt.xlabel(r'Normalized subsystem Size $\frac{l}{L}$')
    plt.ylabel('Entanglement Entropy')
    plt.title('Entanglement Entropy vs. Subsystem Size in the ferromagnetic phase with periodic b.c.')
    plt.legend()
    plt.grid(True)

L_values = [5, 8, 10, 15]
h = 0.3
periodic = True

entropies = compute_entropies(L_values, h, periodic)
plot_results(entropies)
plt.savefig(os.path.join(directory, 'ferro-periodic.png'), dpi =400)
plt.show()


In [ ]:
L_values = range(2, 16)
h = 1.7


Eper=[]
Eopen=[]

for L in L_values:
    if L%2 == 0:
        Eper.append(entanglement_entropy(L//2, L, 1, h, True))
        Eopen.append(entanglement_entropy(L//2, L, 1, h, False))
    else:
        Eper.append(entanglement_entropy(L//2+1, L, 1, h, True))
        Eopen.append(entanglement_entropy(L//2+1, L, 1, h, False))
        
plt.plot(L_values, Eper, label='periodic')
plt.plot(L_values, Eopen, label='open')
plt.xlabel('L')
plt.ylabel('S(L/2; L)')
plt.title('S(L/2; L) vs L in the paramagnetic phase')

plt.legend()
plt.savefig(os.path.join(directory, 'para_SL2.png'), dpi =400)
plt.show()

In [ ]:
L = 15
J = 1
h = 1
periodic = False
ls = np.array(range(1, 15))
entropies=[]

for l in ls:
    entropy=entanglement_entropy(l, L, J, h, periodic)
    entropies.append(entropy)

In [ ]:
def fitfunc(l, c, C):
    return ((c/3) * np.log((L/np.pi) * np.sin(np.pi*(l/L)))) + C

params, covariance = scipy.optimize.curve_fit(fitfunc, ls, entropies)

c, C = params
print('c: ', c)
print('C: ', C)

In [ ]:
lcont = np.linspace(1, 14, 1000)
Econt = fitfunc(lcont, *params)

plt.plot(lcont, Econt, color='xkcd:bright red', label=r'Fitted curve: $S = \frac{{c}}{3} \log\left(\frac{{L}}{{\pi}} \sin\left(\frac{{\pi l}}{{L}}\right)\right) + {C}$')
plt.plot(ls, entropies, 'o', label='Entropies')


plt.xlabel('Segment length l')
plt.ylabel('Entanglement entropy S')
plt.title(f'fit of S for open b.c., L={L} at critical point')
plt.legend()
plt.savefig(os.path.join(directory, 'openfitE.png'), dpi =400)
plt.show()

In [ ]:
def high_entropy(l, L, J, h, periodic):
    
    H = sparseH(L, J, h, periodic)
    
    vec = scipy.sparse.linalg.eigsh(H, k=1, which='LA')[1]
    
    psi_matrix = vec.reshape((2**l, 2**(L-l)))
    u, s, vh = np.linalg.svd(psi_matrix, compute_uv=True)
    lambdas = s**2
    S = -np.sum(lambdas * np.log(lambdas))

    return S

In [ ]:
print(high_entropy(3, 10, 1, 0.4, True))

In [ ]:
def compute_entropies(L_values, h, periodic):
    results = {}
    for L in L_values:
        ls = range(1, L)
        Entropies = []
        for l in tqdm(ls, desc=f'Computing for L={L}'):
            entropy = high_entropy(l, L, 1, h, periodic)
            Entropies.append(entropy)
        results[L] = Entropies
    return results

def plot_results(results):
    plt.figure(figsize=(10, 6))
    for L, entropies in results.items():
        plt.plot(range(1, L)/np.array(L), entropies, label=f'L={L}')
    plt.xlabel(r'Normalized subsystem Size $\frac{l}{L}$')
    plt.ylabel('Entanglement Entropy')
    plt.title('Entanglement Entropy vs. Subsystem Size in the ferromagnetic phase with open b.c.')
    plt.legend()
    plt.grid(True)

L_values = [5, 6, 7, 8, 9, 10, 11, 12, 13]
h = 0.3
periodic = False

entropies = compute_entropies(L_values, h, periodic)
plot_results(entropies)
plt.savefig(os.path.join(directory, 'ferro-openhighest.png'), dpi =400)
plt.show()


In [ ]:
L_values = range(2, 16)
h = 1


Eper=[]
Eopen=[]

for L in L_values:
    if L%2 == 0:
        Eper.append(high_entropy(L//2, L, 1, h, True))
        Eopen.append(high_entropy(L//2, L, 1, h, False))
    else:
        Eper.append(high_entropy(L//2+1, L, 1, h, True))
        Eopen.append(high_entropy(L//2+1, L, 1, h, False))
        
plt.plot(L_values, Eper, label='periodic')
plt.plot(L_values, Eopen, label='open')
plt.xlabel('L')
plt.ylabel('S(L/2; L)')
plt.title('S(L/2; L) vs L at the critical point for the highest excited state')

plt.legend()
plt.savefig(os.path.join(directory, 'criticalSL2.png'), dpi =400)
plt.show()

In [ ]:
L = 14
J = 1
h = 1
periodic = True
ls = np.array(range(1, L))
entropies=[]

for l in ls:
    entropy=high_entropy(l, L, J, h, periodic)
    entropies.append(entropy)

In [ ]:
def fitfunc(l, c, C):
    return ((c/3) * np.log((L/np.pi) * np.sin(np.pi*(l/L)))) + C

params, covariance = scipy.optimize.curve_fit(fitfunc, ls, entropies)

c, C = params
print('c: ', c)
print('C: ', C)

In [ ]:
lcont = np.linspace(1, 13, 1000)
Econt = fitfunc(lcont, *params)

plt.plot(lcont, Econt, color='xkcd:bright red', label=r'Fitted curve: $S = \frac{{c}}{3} \log\left(\frac{{L}}{{\pi}} \sin\left(\frac{{\pi l}}{{L}}\right)\right) + {C}$')
plt.plot(ls, entropies, 'o', label='Entropies')


plt.xlabel('Segment length l')
plt.ylabel('Entanglement entropy S')
plt.title(f'fit of S, periodic b.c., L={L} at critical point for highest state')
plt.legend()
plt.savefig(os.path.join(directory, 'periodicfitEhighestno2.png'), dpi = 400)
plt.show()